In [ ]:
starting_word = "rise" #@param {type:"string", required: true}
genre = "rock" #@param ["rock", "metal", "blues", "pop", "rap"]

# Data preparation and preprocessing

In [ ]:
import tensorflow as tf

import numpy as np
import pandas as pd
import os
import time

device_name = tf.test.gpu_device_name()# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

AUTOTUNE = tf.data.experimental.AUTOTUNE

Found GPU at: /device:GPU:0


## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path_to_file = '/content/drive/MyDrive/Lyrics_generator/english_cleaned_lyrics.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load CSV data
data = pd.read_csv(path_to_file)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Select the columns: lyrics and genre
# columns = ['index', 'song', 'artist', 'year']

# data.drop(columns, inplace=True, axis=1)
#data.drop(data.columns[[0]], axis=1, inplace=True)

data = data[['lyrics', 'genre']]

# Speed up training by taking the first thousand rows (temporary)
data = data[:1000]

#print(lyrics[:10])
#print(genre[:10])
print(data[:10])

                                              lyrics genre
0  Oh baby how you doing You know I'm gonna cut r...   Pop
1  playin everything so easy it's like you seem s...   Pop
2  If you search For tenderness It isn't hard to ...   Pop
3  Oh oh oh I oh oh oh I If I wrote a book about ...   Pop
4  Party the people the people the party it's pop...   Pop
5  I heard Church bells ringing I heard A choir s...   Pop
6  This is just another day that I would spend Wa...   Pop
7  Waiting waiting waiting waiting Waiting waitin...   Pop
8   I read all of the magazines while waiting aro...   Pop
9  N n now honey You better sit down and look aro...   Pop


## Preprocess and split dataset

In [ ]:
# Lower
def lyrics_preprocessing(lyrics_text):
    return lyrics_text.lower() #.split()

data['lyrics'] = data['lyrics'].apply(lyrics_preprocessing)

# One Hot encoding
data = pd.get_dummies(data, columns=['genre'])

#print(data[:10])

lyrics = data[['lyrics']]
genres = data.iloc[:, 1:]

print(lyrics[:10])
print(genres[:10])

print(genres.shape)
genres_size = genres.shape[1]

                                              lyrics
0  oh baby how you doing you know i'm gonna cut r...
1  playin everything so easy it's like you seem s...
2  if you search for tenderness it isn't hard to ...
3  oh oh oh i oh oh oh i if i wrote a book about ...
4  party the people the people the party it's pop...
5  i heard church bells ringing i heard a choir s...
6  this is just another day that i would spend wa...
7  waiting waiting waiting waiting waiting waitin...
8   i read all of the magazines while waiting aro...
9  n n now honey you better sit down and look aro...
   genre_Country  genre_Electronic  genre_Folk  genre_Hip-Hop  genre_Indie  \
0              0                 0           0              0            0   
1              0                 0           0              0            0   
2              0                 0           0              0            0   
3              0                 0           0              0            0   
4              0           

# NUOVA VERSIONE TF


In [ ]:
def split_l(x):
  splitted = x.split()[:6]
  if len(splitted) < 6: 
    return None
  else:
    return " ".join(splitted)

'''nan_values = l.isna()
nan_columns = nan_values.any()
print("Prima: ", nan_columns)'''

l = data['lyrics'].apply(split_l)
l.dropna(axis=0, inplace=True)

'''nan_values = l.isna()
nan_columns = nan_values.any()
print("Dopo: ", nan_columns)'''
#print("Dopo: ", l.shape)

tokens = tf.strings.split(l)

#tokens = data['lyrics'].apply(lambda t: tf.strings.split(t, maxsplit=5))

tokens[:10]

<tf.RaggedTensor [[b'oh', b'baby', b'how', b'you', b'doing', b'you'], [b'playin', b'everything', b'so', b'easy', b"it's", b'like'], [b'if', b'you', b'search', b'for', b'tenderness', b'it'], [b'oh', b'oh', b'oh', b'i', b'oh', b'oh'], [b'party', b'the', b'people', b'the', b'people', b'the'], [b'i', b'heard', b'church', b'bells', b'ringing', b'i'], [b'this', b'is', b'just', b'another', b'day', b'that'], [b'waiting', b'waiting', b'waiting', b'waiting', b'waiting', b'waiting'], [b'i', b'read', b'all', b'of', b'the', b'magazines'], [b'n', b'n', b'now', b'honey', b'you', b'better']]>

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

#lyrics_tensor = tf.convert_to_tensor(data['lyrics'].tolist())

layer = StringLookup()
layer.adapt(tokens)
vocab = layer.get_vocabulary()

vocab[:10]

['', '[UNK]', 'the', 'i', 'you', 'a', 'to', 'in', 'and', 'my']

In [ ]:
ids_from_words = layer(tokens)
#ids_from_words = StringLookup(vocabulary=list(vocab))


ids_from_words[:2]
#ids = ids_from_words(tokens)
#ids[:2]

<tf.RaggedTensor [[17, 36, 70, 4, 556, 4], [937, 214, 41, 216, 27, 30]]>

In [ ]:
words_from_ids = StringLookup(
    vocabulary=layer.get_vocabulary(), invert=True)
#words_from_ids = StringLookup(
#    vocabulary=ids_from_words.get_vocabulary(), invert=True)

words = words_from_ids(ids_from_words)
#words = words_from_ids(ids)
words[:10]

<tf.RaggedTensor [[b'oh', b'baby', b'how', b'you', b'doing', b'you'], [b'playin', b'everything', b'so', b'easy', b"it's", b'like'], [b'if', b'you', b'search', b'for', b'tenderness', b'it'], [b'oh', b'oh', b'oh', b'i', b'oh', b'oh'], [b'party', b'the', b'people', b'the', b'people', b'the'], [b'i', b'heard', b'church', b'bells', b'ringing', b'i'], [b'this', b'is', b'just', b'another', b'day', b'that'], [b'waiting', b'waiting', b'waiting', b'waiting', b'waiting', b'waiting'], [b'i', b'read', b'all', b'of', b'the', b'magazines'], [b'n', b'n', b'now', b'honey', b'you', b'better']]>

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(words_from_ids(ids), axis=-1, separator=' ')

In [ ]:
#seq_length = 32

#sequences = ids_from_words.batch(seq_length, drop_remainder=True)

sequences = tf.data.Dataset.from_tensor_slices(ids_from_words)
#sequences = tf.data.Dataset.from_tensor_slices(ids)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'oh baby how you doing'
Target: b'baby how you doing you'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, None), (64, None)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [ ]:
### GENERATOR

class Generator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units, dense_size):
      super().__init__(self)
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
      self.gru = tf.keras.layers.GRU(rnn_units,
                                    return_sequences=True, 
                                    return_state=True)
      self.dense = tf.keras.layers.Dense(dense_size)

  def call(self, inputs, states=None, return_state=False, training=False):
      x = inputs
      
      print("gen - shape before embedding: ", x.shape)
      x = self.embedding(x, training=training)
      print("gen - shape after embedding: ", x.shape)
      print("gen - shape before gru: ", x.shape)
      if states is None:
        states = self.gru.get_initial_state(x)
      x, states = self.gru(x, initial_state=states, training=training)

      print("gen - shape after gru: ", x.shape)
      # Dense layer with vocab_size + genres_size 

      print("gen - shape before dense: ", x.shape)
      x = self.dense(x, training=training)
      print("gen - shape after dense: ", x.shape)

      if return_state:
        return x, states
      else: 
        return x

In [ ]:
### DISCRIMINATOR

class Discriminator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units, hidden_size):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.flatten = tf.keras.layers.Flatten()
    self.dense_one = tf.keras.layers.Dense(hidden_size)
    self.dense_two = tf.keras.layers.Dense(1)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs

    print("dis - shape before embedding: ", x.shape)
    x = self.embedding(x, training=training)
    print("dis - shape after embedding: ", x.shape)
    print("dis - shape before gru: ", x.shape)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)

    print("dis - shape after gru: ", x.shape)
    print("dis - shape before flatten: ", x.shape)
    x = self.flatten(x)
    print("dis - shape after flatten: ", x.shape)

    # Dense layers
    print("dis - shape before dense: ", x.shape)
    x = self.dense_one(x, training=training)
    x = self.dense_two(x, training=training)
    print("dis - shape after dense: ", x.shape)

    print("dis - shape before sigmoid: ", x.shape)
    x = tf.keras.activations.sigmoid(x)
    print("dis - shape after sigmoid: ", x.shape)

    if return_state:
      return x, states
    else: 
      return x

In [ ]:
vocab_size = len(vocab)

gen = Generator(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    #dense_size=vocab_size + genres_size
    dense_size=vocab_size)

dis = Discriminator(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    hidden_size=32)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions_gen = gen(input_example_batch)
    #print(example_batch_predictions_gen, "# example_batch_predictions_gen")
    print(example_batch_predictions_gen.shape, "# (batch_size, sequence_length, vocab_size)")

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions_dis = dis(input_example_batch)
    #print(example_batch_predictions_dis, "# example_batch_predictions_dis")
    print(example_batch_predictions_dis.shape, "# (batch_size, output_size)")

dis_fake_target = tf.fill(BATCH_SIZE, 0.0)
dis_real_target = tf.fill(BATCH_SIZE, 1.0)

gen - shape before embedding:  (64, 5)
gen - shape after embedding:  (64, 5, 256)
gen - shape before gru:  (64, 5, 256)
gen - shape after gru:  (64, 5, 1024)
gen - shape before dense:  (64, 5, 1024)
gen - shape after dense:  (64, 5, 1520)
(64, 5, 1520) # (batch_size, sequence_length, vocab_size)
dis - shape before embedding:  (64, 5)
dis - shape after embedding:  (64, 5, 256)
dis - shape before gru:  (64, 5, 256)
dis - shape after gru:  (64, 5, 1024)
dis - shape before flatten:  (64, 5, 1024)
dis - shape after flatten:  (64, 5120)
dis - shape before dense:  (64, 5120)
dis - shape after dense:  (64, 1)
dis - shape before sigmoid:  (64, 1)
dis - shape after sigmoid:  (64, 1)
(64, 1) # (batch_size, output_size)


In [ ]:
gen.summary()

Model: "generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  389120    
_________________________________________________________________
gru_2 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_3 (Dense)              multiple                  1558000   
Total params: 5,885,424
Trainable params: 5,885,424
Non-trainable params: 0
_________________________________________________________________


In [ ]:
dis.summary()

Model: "discriminator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  389120    
_________________________________________________________________
gru_3 (GRU)                  multiple                  3938304   
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple                  163872    
_________________________________________________________________
dense_5 (Dense)              multiple                  33        
Total params: 4,491,329
Trainable params: 4,491,329
Non-trainable params: 0
_________________________________________________________________


## Training



In [ ]:
# define loss functions and optimizers
gen_loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
dis_loss = tf.losses.BinaryCrossentropy()

gen_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
dis_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
example_batch_loss = gen_loss(target_example_batch, example_batch_predictions_gen)
mean_gen_loss = example_batch_predictions_gen.numpy().mean()

print("Prediction shape: ", example_batch_predictions_gen.shape, " # (batch_size, sequence_lenght, vocab_size)")
print("Mean loss:        ", mean_gen_loss)

Prediction shape:  (64, 5, 1520)  # (batch_size, sequence_lenght, vocab_size)
Mean loss:         1.4960917e-05


In [ ]:
example_batch_loss = dis_loss(dis_real_target, example_batch_predictions_dis)
mean_dis_loss = example_batch_predictions_dis.numpy().mean()

print("Prediction shape: ", example_batch_predictions_dis.shape, " # (batch_size, output_size)")
print("Mean loss:        ", mean_dis_loss)

Prediction shape:  (64, 1)  # (batch_size, output_size)
Mean loss:         0.50037855


In [ ]:
print(tf.exp(mean_gen_loss).numpy())
print(tf.exp(mean_dis_loss).numpy())

1.000015
1.6493455


In [ ]:
#gen.compile(optimizer='adam', loss=gen_loss)
#dis.compile(optimizer='adam', loss=dis_loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [ ]:
def loss(model, x, y, training, loss_fn):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)

  return y_, loss_fn(y_true=y, y_pred=y_)


y_gen, gen_l = loss(gen, input_example_batch, target_example_batch, training=False, loss_fn=gen_loss)
print("Gen loss test: {}".format(gen_l))

y_dis, dis_l = loss(dis, input_example_batch, dis_real_target, training=False, loss_fn=dis_loss)
print("Dis loss test: {}".format(dis_l))

gen - shape before embedding:  (64, 5)
gen - shape after embedding:  (64, 5, 256)
gen - shape before gru:  (64, 5, 256)
gen - shape after gru:  (64, 5, 1024)
gen - shape before dense:  (64, 5, 1024)
gen - shape after dense:  (64, 5, 1520)
Gen loss test: 7.327142238616943
dis - shape before embedding:  (64, 5)
dis - shape after embedding:  (64, 5, 256)
dis - shape before gru:  (64, 5, 256)
dis - shape after gru:  (64, 5, 1024)
dis - shape before flatten:  (64, 5, 1024)
dis - shape after flatten:  (64, 5120)
dis - shape before dense:  (64, 5120)
dis - shape after dense:  (64, 1)
dis - shape before sigmoid:  (64, 1)
dis - shape after sigmoid:  (64, 1)
Dis loss test: 0.6924004554748535


In [ ]:
def grad(model, inputs, targets, loss_fn):
  with tf.GradientTape() as tape:
    y_, loss_value = loss(model, inputs, targets, training=True, loss_fn=loss_fn)
  return y_, loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
def generate_one_step(inputs, ids_from_words, temperature=1.0):
  # Create a mask to prevent "" or "[UNK]" from being generated.
  skip_ids = ids_from_words(['','[UNK]'])[:, None]
  sparse_mask = tf.SparseTensor(
      # Put a -inf at each bad index.
      values=[-float('inf')]*len(skip_ids),
      indices = skip_ids,
      # Match the shape to the vocabulary
      dense_shape=[len(ids_from_words.get_vocabulary())]) 
      # Shape to the vocabulary + genres
      #dense_shape=[len(ids_from_words.get_vocabulary()) + genres_size]) 
  prediction_mask = tf.sparse.to_dense(sparse_mask)

  #print("generate_one_step - inputs: ", inputs.shape) # (batch_size, sequence_lenght, vocab_size)


  # Only use the last prediction.
  predicted_logits = inputs[:, -1, :]
  predicted_logits = predicted_logits/temperature
  # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
  predicted_logits = predicted_logits + prediction_mask

  # Sample the output logits to generate token IDs.
  predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
  predicted_ids = tf.squeeze(predicted_ids, axis=-1)

  # Return the predicted ids
  return predicted_ids

  '''
  # Define a list of predicted IDs tensors (sequence_length tensors of batch_size items)
  predicted_ids_matrix = []
  
  batch_size, sequence_length, _ = inputs.shape

  # Predict IDs for all the predicted logits of a sequence in a batch
  for i in range(sequence_length):
    # Only use the i-th prediction
    predicted_logits = inputs[:, i, :]
    print("generate_one_step - inputs[:, " + str(i) + ", :]: ", predicted_logits.shape) # (batch_size, vocab_size)

    predicted_logits = predicted_logits/temperature
    print("generate_one_step - inputs[:, " + str(i) + ", :]: ", predicted_logits.shape) # (batch_size, vocab_size)

    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + prediction_mask
    print("generate_one_step - predicted_logits + prediction_mask: ", predicted_logits.shape) # (batch_size, vocab_size)

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    print("generate_one_step - predicted_ids: ", predicted_ids.shape) # (batch_size, 1)

    # Append the predicted IDs tensor to the list
    predicted_ids_matrix.append(predicted_ids) 


  # predicted_ids_matrix now includes sequence_length tensors of batch_size items
  # We want batch_size tensors of sequence_length items by grouping element on the axis 1!

  #print("predicted_ids_matrix: ", predicted_ids_matrix)
  predicted_ids_matrix = tf.stack(predicted_ids_matrix, axis=1)
  #print("predicted_ids_matrix stack: ", predicted_ids_matrix)
  predicted_ids_matrix = tf.squeeze(predicted_ids_matrix, axis=-1)
  #print("predicted_ids_matrix squeeze: ", predicted_ids_matrix)
  print("generate_one_step - predicted_ids_matrix.shape: ", predicted_ids_matrix.shape) # (batch_size, sequence_lenght)

  # Return the predicted IDs matrix 
  return predicted_ids_matrix'''


In [ ]:
from tqdm import tqdm

SEQUENCE_LENGTH = 5

def train(gen, dis, dataset, epochs=20):
  train_loss_results_gen = []
  train_loss_results_dis = []
  train_accuracy_results_gen = []

  for epoch in range(epochs):
    epoch_loss_avg_gen = tf.keras.metrics.Mean()
    epoch_accuracy_gen = tf.keras.metrics.SparseCategoricalAccuracy()

    epoch_loss_avg_dis = tf.keras.metrics.Mean()
    epoch_accuracy_dis = tf.keras.metrics.SparseCategoricalAccuracy()

    epoch_num = 1

    for input_batch, target_batch in tqdm(dataset):
      '''print("\n********************************\n\nepoch: ", epoch_num)
      #print("input_batch: ", input_batch)
      y_gen, gen_l, gen_grad = grad(gen, input_batch, target_batch, gen_loss)

      gen_optimizer.apply_gradients(zip(gen_grad, gen.trainable_variables))'''




      #gen_l    = 0
      #gen_grad = 0

      # Define a list of predicted IDs tensors (sequence_length tensors of batch_size items)
      y_gen_seq = input_batch.numpy()
      #print("y_gen_seq: ", y_gen_seq)

      for i in range(SEQUENCE_LENGTH):
        # If first iteration, than train
        if i == 0:
          print("\n********************************\n\nepoch: ", epoch_num)
          #print("input_batch: ", input_batch)
          y_gen, gen_l, gen_grad = grad(gen, input_batch, target_batch, gen_loss)

          gen_optimizer.apply_gradients(zip(gen_grad, gen.trainable_variables))
        # Else generate other ids to fullfill a sequence
        else:
          #print("tf.stack(y_gen_seq).shape: ", tf.stack(y_gen_seq).shape)
          y_gen = gen(tf.stack(y_gen_seq), training=False)

        print("\ny_gen.shape before one_step: ", y_gen.shape)
        y_gen = generate_one_step(y_gen, layer)
        print("y_gen.shape after one_step: ", y_gen.shape)

        # Remove head of each sequence
        y_gen_seq = np.delete(y_gen_seq, 0, axis=1)
        #print("y_gen_seq after delete: ", y_gen_seq)
        #print("tf.expand_dims(y_gen, axis=-1): ", tf.expand_dims(y_gen, axis=-1))
        # Append the predicted IDs array to the list
        y_gen_seq = np.append(y_gen_seq, tf.expand_dims(y_gen, axis=-1), axis=1)
        #print("y_gen_seq: ", y_gen_seq)

      # target_batch.shape must be equal to y_gen_seq.shape!
      print("\ntarget_batch: ", target_batch)
      print("\ntarget_batch.shape: ", target_batch.shape)
      print("y_gen_seq: ", y_gen_seq)
      print("y_gen_seq.shape: ", y_gen_seq.shape)


      print()
      y_dis_fake, dis_fake_l, dis_fake_grad = grad(dis, y_gen_seq, dis_fake_target, dis_loss)
      print()
      #y_dis_real, dis_real_l, dis_real_grad = grad(dis, input_batch, dis_real_target, dis_loss)
      y_dis_real, dis_real_l, dis_real_grad = grad(dis, target_batch, dis_real_target, dis_loss)

      # Wasserstein loss
      dis_l = tf.reduce_mean(y_dis_fake) - tf.reduce_mean(y_dis_real)
      print("dis_l: ", dis_l)

      alpha = tf.random.uniform(
          shape=[tf.shape(target_batch)[0], 5],
          minval=0.,
          maxval=1.
      )
      print("\nalpha.shape: ", alpha.shape)

      '''differences = y_dis_fake - y_dis_real
      print("differences.shape: ", differences.shape)
      interpolates = y_dis_real + (alpha * differences)
      print("interpolates.shape: ", interpolates.shape)'''

      # Cast to float to calculate the interpolation
      y_gen_seq    = tf.cast(y_gen_seq, tf.float32)
      target_batch = tf.cast(target_batch, tf.float32)

      differences = y_gen_seq - target_batch
      print("differences: ", differences)
      print("differences.shape: ", differences.shape)
      interpolates = target_batch + (alpha * differences)
      print("interpolates: ", interpolates)
      print("interpolates.shape: ", interpolates.shape)
      
      # For successive discriminator embedding
      #interpolates = tf.cast(interpolates, tf.int64)
      #print("interpolates: ", interpolates)

      with tf.GradientTape() as tape:
        '''y_, loss_value = loss(dis, inputs, targets, training=False, loss_fn=dis_loss)
        return y_, loss_value, tape.gradient(loss_value, model.trainable_variables)'''

        #y_interpolates = dis(interpolates)
        #print("y_interpolates: ", y_interpolates)
        #gradients = tape.gradient(dis(interpolates), [interpolates])[0]
        #gradients = tape.gradient(dis(interpolates), interpolates)

        gradients = tape.gradient(dis(interpolates), dis.trainable_variables)        
        #gradients = tape.gradient(dis_l, dis.trainable_variables)
        
        print("gradients: ", gradients)
  
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1, 2]))
        gradient_penalty = tf.reduce_mean((slopes - 1.) ** 2)
        dis_grad = dis_l + 10 * gradient_penalty

        dis_optimizer.apply_gradients(zip(dis_grad, dis.trainable_variables))

      # Track progress
      epoch_loss_avg_gen.update_state(gen_l)
      epoch_loss_avg_dis.update_state(dis_l)  
      # Compare predicted label to actual label
      # training=True is needed only if there are layers with different
      # behavior during training versus inference (e.g. Dropout).
      epoch_accuracy_gen.update_state(target_batch, y_gen)
      #epoch_accuracy_dis.update_state(y, model(x))
      epoch_accuracy_dis.update_state(dis_real_target, y_dis_real)
      epoch_accuracy_dis.update_state(dis_fake_target, y_dis_fake)

      epoch_num += 1

    # End epoch
    train_loss_results_gen.append(epoch_loss_avg_gen.result())
    train_loss_results_dis.append(epoch_loss_avg_dis.result())
    train_accuracy_results_gen.append(epoch_accuracy_gen.result())

    print("\nEpoch {:03d}: Loss gen: {:.3f}, Loss dis: {:.3f}, Accuracy gen: {:.3%}".format(epoch,
                                                                  epoch_loss_avg_gen.result(),
                                                                  3, ##epoch_loss_avg_dis.result(),
                                                                  epoch_accuracy_gen.result()))

In [ ]:
EPOCHS = 30 #20
train(gen, dis, dataset, EPOCHS)

  0%|          | 0/15 [00:00<?, ?it/s]


********************************

epoch:  1
gen - shape before embedding:  (64, 5)
gen - shape after embedding:  (64, 5, 256)
gen - shape before gru:  (64, 5, 256)
gen - shape after gru:  (64, 5, 1024)
gen - shape before dense:  (64, 5, 1024)
gen - shape after dense:  (64, 5, 1520)

y_gen.shape before one_step:  (64, 5, 1520)
y_gen.shape after one_step:  (64,)
gen - shape before embedding:  (64, 5)
gen - shape after embedding:  (64, 5, 256)
gen - shape before gru:  (64, 5, 256)
gen - shape after gru:  (64, 5, 1024)
gen - shape before dense:  (64, 5, 1024)
gen - shape after dense:  (64, 5, 1520)

y_gen.shape before one_step:  (64, 5, 1520)
y_gen.shape after one_step:  (64,)
gen - shape before embedding:  (64, 5)
gen - shape after embedding:  (64, 5, 256)
gen - shape before gru:  (64, 5, 256)
gen - shape after gru:  (64, 5, 1024)
gen - shape before dense:  (64, 5, 1024)
gen - shape after dense:  (64, 5, 1520)

y_gen.shape before one_step:  (64, 5, 1520)
y_gen.shape after one_step:  (64,

ValueError: ignored

In [ ]:
#EPOCHS = 20

#history = gen.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, words_from_ids, ids_from_words, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.words_from_ids = words_from_ids
    self.ids_from_words = ids_from_words

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_words(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_words.get_vocabulary())]) 
        # Shape to the vocabulary + genres
        #dense_shape=[len(ids_from_words.get_vocabulary()) + genres_size]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    '''input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_words(input_chars).to_tensor())'''
    input_ids = self.ids_from_words([inputs])

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, 
                                           states=states, 
                                           return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_words = self.words_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_words, states

In [ ]:
#one_step_model = OneStep(model, words_from_ids, ids_from_words)
#one_step_model = OneStep(model, words_from_ids, layer)

one_step_model = OneStep(gen, words_from_ids, layer)

In [ ]:
start  = time.time()
states = None
next_word = tf.constant(['romeo'])
result    = [next_word]

for n in range(100):
  next_word, states = one_step_model.generate_one_step(next_word, states=states)
  result.append(next_word)

print(tf.strings.join(result, separator=" ")[0].numpy().decode("utf-8"))
end    = time.time()

print(f"\nRun time: {end - start}")


In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['love'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result, separator=" ")[0].numpy().decode("utf-8"))

# VECCHIA VERSIONE TF

## Vectorize the lyrics

In [ ]:
# Build the Vectorizer
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Init
vectorize_layer = TextVectorization(
    output_mode='int',
    output_sequence_length=50)

# Load data as TF Dataset object
tf_lyrics = tf.data.Dataset.from_tensor_slices(lyrics)
tf_genre = tf.data.Dataset.from_tensor_slices(genre)

# Fit the Vectorizer
vectorize_layer.adapt(tf_lyrics.batch(1024))
inverse_vocab = vectorize_layer.get_vocabulary()

ids_from_chars = StringLookup(
    vocabulary=inverse_vocab)

# Traform (vectorize) the lyrics
def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return tf.squeeze(vectorize_layer(text))

# Vectorize the data in text_ds.
tf_vec_lyrics = tf_lyrics.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer)

ids = ids_from_chars(tf_vec_lyrics)



In [ ]:
# Build the Vectorizer
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Init
vectorize_layer = TextVectorization(
    output_mode='int',
    output_sequence_length=50)

# Load data as TF Dataset object
tf_lyrics = tf.data.Dataset.from_tensor_slices(lyrics)
tf_genre = tf.data.Dataset.from_tensor_slices(genre)

# Fit the Vectorizer
vectorize_layer.adapt(tf_lyrics.batch(1024))

# Save the inverse vocabulary
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

In [ ]:
'''
# Traform (vectorize) the lyrics
def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return tf.squeeze(vectorize_layer(text))

# Vectorize the data in text_ds.
tf_vec_lyrics = tf_lyrics.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

# Print the vectorized text
sequences = list(tf_vec_lyrics.as_numpy_iterator())
             
print(len(sequences))

#for seq in sequences[:5]:
#  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")
'''

In [ ]:
# Transform (vectorize) the lyrics
def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return tf.squeeze(vectorize_layer(text))

# Vectorize the data in text_ds.
tf_vec_lyrics = tf_lyrics.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

# Print the vectorized text
sequences = tf_vec_lyrics.batch(seq_length+1, drop_remainder=True)

#print(len(sequences))

for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

In [ ]:
ids_from_terms = preprocessing.StringLookup(
    vocabulary=list(inverse_vocab))

ids = ids_from_terms(tf_lyrics)

terms_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_terms.get_vocabulary(), invert=True)

terms = terms_from_ids(ids)

tf.strings.reduce_join(terms, axis=-1).numpy()



def text_from_ids(term):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
def split_input_target(lyrics):
    input_text = lyrics[:-1]
    target_text = lyrics[1:]
    return input_text, target_text

In [ ]:
tf_vec_lyrics = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in tf_vec_lyrics.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

In [ ]:
'''
ids_from_terms = preprocessing.StringLookup(
    vocabulary=list(inverse_vocab))

ids = ids_from_terms(terms)

chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

chars = chars_from_ids(ids)
'''

In [ ]:
# The maximum length sentence you want for a single input in characters
#seq_length = 10
#examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
#char_dataset = tf.data.Dataset.from_tensor_slices(list(text_vector_ds.as_numpy_iterator())[0])

In [ ]:
#sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

#def split_input_target(chunk):
#    input_text = chunk[:-1]
#    target_text = chunk[1:]
#    return input_text, target_text

#dataset = sequences.map(split_input_target)

## Build the model

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

tf_vec_lyrics = tf_vec_lyrics.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(inverse_vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(inverse_vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [ ]:
'''
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size=len(inverse_vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
'''

In [ ]:
#example_batch_predictions = 0

#for input_example_batch, target_example_batch in tf_vec_lyrics.take(1):
#    example_batch_predictions = model(input_example_batch)
#    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

#model.summary()


In [ ]:
tf_vec_lyrics.batch(50, drop_remainder = True)

for lyric in tf_vec_lyrics.take(1):
  example_batch_predictions = model(lyric)

model.summary()

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())
